# Training des ML-Algorithmus

## Laden

In [1]:
import pickle as cPickle
import csv
import random
import numpy as np
import copy
from sklearn.feature_extraction.text import CountVectorizer
import time
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, precision_score, recall_score
import pandas as pd
from IPython.display import clear_output
from math import pi
from sklearn.metrics import confusion_matrix

np.random.seed(31415)

with open('combined_malicious_and_normal.csv', 'r') as csvfile:
    csvtext = csvfile.readlines()
    
datensatz = []
for line in csvtext:
    datensatz.append(tuple(line.strip().split(', '))[0])
    
print(datensatz[:3])

['www.api.binance.com', 'google.com', 'pool.burstcoin.ml']


# Ergebnisse wieder anfügen für jeden Eintrag der Liste
## Zugehörigkeit speichern (maliziös|ok)
### ok = 1, maliziös = 0

In [2]:
#Der erste Eintrag der kombinierten Liste ist maliziös. Ab da an alternierend.
#0 = malicious
#1 = good

y = []
for i in range(len(datensatz)):
    if(i%2 == 0):
       y.append("malicious")
    else:
        y.append("ok")
#print(y[:3])
#print(X[:3])

## Datensatz aufteilen

In [3]:
def filter_numbers(data):
    function_data = copy.copy(data)
    function_data = [item.replace('0', '1') for item in function_data]
    function_data = [item.replace('2', '1') for item in function_data]
    function_data = [item.replace('3', '1') for item in function_data]
    function_data = [item.replace('4', '1') for item in function_data]
    function_data = [item.replace('5', '1') for item in function_data]
    function_data = [item.replace('6', '1') for item in function_data]
    function_data = [item.replace('7', '1') for item in function_data]
    function_data = [item.replace('8', '1') for item in function_data]
    function_data = [item.replace('9', '1') for item in function_data]
    return function_data

def reduce_dataset(dataset_x, dataset_y, count,start_number=0):
    X_temp = copy.copy(dataset_x)
    y_temp = copy.copy(dataset_y)
    
    last_number = count+start_number
    if last_number > len(X_temp):
        last_number = len(X_temp)
    
    X_temp_i = X_temp[start_number:last_number:1]
    y_temp_i = y_temp[start_number:last_number:1]   
    return X_temp_i,y_temp_i

X = datensatz

print(X[:14])
print(str(len(X)))
X, y = reduce_dataset(X, y, 500000,0)
print(str(len(X)))

print(X[:14])
print(y[:14])

['www.api.binance.com', 'google.com', 'pool.burstcoin.ml', 'youtube.com', 'www.uupool.com', 'tmall.com', 'mollnia.com', 'baidu.com', 'wn-iad02.e-planning.net', 'qq.com', 'jicfbcczdh.com', 'sohu.com', 'srv-2014-07-18-00.config.parsely.com', 'facebook.com']
642842
500000
['www.api.binance.com', 'google.com', 'pool.burstcoin.ml', 'youtube.com', 'www.uupool.com', 'tmall.com', 'mollnia.com', 'baidu.com', 'wn-iad02.e-planning.net', 'qq.com', 'jicfbcczdh.com', 'sohu.com', 'srv-2014-07-18-00.config.parsely.com', 'facebook.com']
['malicious', 'ok', 'malicious', 'ok', 'malicious', 'ok', 'malicious', 'ok', 'malicious', 'ok', 'malicious', 'ok', 'malicious', 'ok']


## URL's vektorisieren

In [4]:
def vectorize(n_gram_min, n_gram_max, data, amount_of_params, vectorizer = None):
    #dafür sorgen dass der Ursprungsdatensatz nicht korrumpiert wird
    data = copy.deepcopy(data)
    
    #Initialisieren und Dictionary anlegen
    if vectorizer == None:
        vectorizer = CountVectorizer(analyzer='word', token_pattern=r'[^\s+]', max_features = amount_of_params, ngram_range=(n_gram_min, n_gram_max))
        start_time_fit = time.time()
        vectorizer.fit(data)
        end_time_fit = time.time()
        time_for_fit = end_time_fit - start_time_fit
        #print("time_for_fit:" + str(time_for_fit))
    
    #Dictionary anwenden und N-Gram Vektorisierng durchführen
    start_time_transform = time.time()
    data = vectorizer.transform(data)
    end_time_transform = time.time()
    time_for_transform = end_time_transform - start_time_transform
    
    
    #print("time_for_transform:" + str(time_for_transform))   
    return data, vectorizer

# Blacklist Überprüfungs-Logik

In [5]:
def domain_is_malicious(blacklist, domain_name):
    is_malicious = False
    if (domain_name in blacklist):
        is_malicious = True
    else:
        is_malicious = False
    return is_malicious

# Bearbeitung der Szenarien

## Parameter des Random Forest festlegen

In [6]:
n_gram_min = 0
n_gram_max = 4
max_param_amount = 726
test_size_percentage = .11
tree_count = 9

## Szenario 1: 30% bekannt, 70% unbekannt

### Ursprungsdatenmenge und Testdatenmenge aufteilen

In [7]:
groesse_der_bekannten_menge = 0.3

print("Vollständige Datensatzanzahl: " + str(len(X)))
Vollstaendige_Anzahl = len(X)
bekannte_Anzahl = round(Vollstaendige_Anzahl*groesse_der_bekannten_menge)
unbekannte_Anzahl = Vollstaendige_Anzahl - bekannte_Anzahl

Vollständige Datensatzanzahl: 500000


## Zufallsverfahren

In [8]:
#Auf dem gemischten Datensatz evaluieren
TP = 0
TN = 0
FP = 0
FN = 0
for i in range(len(X)):
    predicted_is_malicious = random.choice([True, False])
    real_is_malicious = y[i]
    if real_is_malicious == "malicious":
        real_is_malicious = True
    else:
        real_is_malicious = False
    
    if real_is_malicious == True:
        if predicted_is_malicious == True:
            TP += 1
        elif predicted_is_malicious == False:
            FN += 1
    elif real_is_malicious == False:
        if predicted_is_malicious == True:
            FP += 1   
        elif predicted_is_malicious == False:
            TN += 1
 
print("TP: " + str(TP))
print("TN: " + str(TN))
print("FP: " + str(FP))
print("FN: " + str(FN))
accuracy = (TP + TN) / (TP + TN + FP + FN)
print("Accuracy auf der gemischten Datenmenge: " + str(accuracy))

TP: 125279
TN: 125394
FP: 124606
FN: 124721
Accuracy auf der gemischten Datenmenge: 0.501346


### Random Forest Test durchführen

In [9]:
X_rf_filtered_numbers = filter_numbers(X)
Xbekannt, ybekannt = reduce_dataset(X_rf_filtered_numbers, y, bekannte_Anzahl,0)
Xunbekannt, yunbekannt = reduce_dataset(X_rf_filtered_numbers, y, unbekannte_Anzahl,bekannte_Anzahl)
print("Anzahl bekannter zu unbekannter Datensätze:")
print(str(len(Xbekannt)))
print(str(len(Xunbekannt)))
#Training nur innerhalb der Ursprungsdatenmenge mit der zuvor ermittelten Test-Size-Größe
vectorized_Xbekannt, vectorizer = vectorize(n_gram_min, n_gram_max, Xbekannt, max_param_amount)
X_train, X_test, y_train, y_test = train_test_split(vectorized_Xbekannt, ybekannt, test_size=test_size_percentage, random_state=0)
classifier = RandomForestClassifier(n_estimators=tree_count, random_state=0,n_jobs=-1)
start_time_fit = time.time()
classifier.fit(X_train, y_train) 
end_time_fit = time.time()
time_fit = end_time_fit - start_time_fit 
print("time_to_fit:" + str(time_fit))
print("---------")

y_pred = classifier.predict(X_train)
accuracy = accuracy_score(y_train, y_pred)
print("Accuracy der Trainingsmenge innerhalb der bekannten Menge: " + str(accuracy))
CM = confusion_matrix(y_train, y_pred)

print("---------")
TN = CM[0][0]
FN = CM[1][0]
TP = CM[1][1]
FP = CM[0][1]
print("TN: " + str(TN))
print("FN: " + str(FN))
print("TP: " + str(TP))
print("FP: " + str(FP))

print("---------")

y_pred = classifier.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy der Validationsmenge innerhalb der bekannten Menge: " + str(accuracy))
CM = confusion_matrix(y_test, y_pred)
print("Accuracy:" + str(accuracy))
print("---------")
TN = CM[0][0]
FN = CM[1][0]
TP = CM[1][1]
FP = CM[0][1]
print("TN: " + str(TN))
print("FN: " + str(FN))
print("TP: " + str(TP))
print("FP: " + str(FP))

print("---------")

#Unbekannte Menge bearbeiten..
vectorized_Xunbekannt, vectorizer = vectorize(n_gram_min, n_gram_max, Xunbekannt, max_param_amount, vectorizer)
X_train, X_test, y_train, y_test = train_test_split(vectorized_Xunbekannt, yunbekannt, test_size=test_size_percentage, random_state=0)
y_pred = classifier.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy auf der unbekannten Menge: " + str(accuracy))

#Memory freigeben
del vectorized_Xbekannt, vectorized_Xunbekannt, X_train, X_test, y_train, y_test, y_pred

#Beides zusammen bearbeiten für das zu ermittelnde Ergebnis
vectorized_Xmix, vectorizer = vectorize(n_gram_min, n_gram_max, X_rf_filtered_numbers, max_param_amount, vectorizer)
#X_train, X_test, y_train, y_test = train_test_split(vectorized_Xmix, y, test_size=test_size_percentage, random_state=0)
start_time_predict = time.time()
y_pred = classifier.predict(vectorized_Xmix)
end_time_predict = time.time()
time_predict = end_time_predict - start_time_predict 
print("time_to_prediction:" + str(time_predict))

accuracy = accuracy_score(y, y_pred)
print("Accuracy auf der vollständigen gemischten Menge: " + str(accuracy))
CM = confusion_matrix(y, y_pred)
print("Accuracy:" + str(accuracy))
print("---------")
TN = CM[0][0]
FN = CM[1][0]
TP = CM[1][1]
FP = CM[0][1]
print("TN: " + str(TN))
print("FN: " + str(FN))
print("TP: " + str(TP))
print("FP: " + str(FP))
print("---------")

print("Sicherheitstest..:")
vectorized_google, vectorizer = vectorize(n_gram_min, n_gram_max, ["google.de"], max_param_amount, vectorizer)
print("google.de: " + str(classifier.predict(vectorized_google)))
vectorized_1111, vectorizer = vectorize(n_gram_min, n_gram_max, ["1111-c111-1b11-a11d.reporo.net"], max_param_amount, vectorizer)
print("1111-c111-1b11-a11d.reporo.net: " + str(classifier.predict(vectorized_1111)))

Anzahl bekannter zu unbekannter Datensätze:
150000
350000
time_to_fit:16.098351001739502
---------
Accuracy der Trainingsmenge innerhalb der bekannten Menge: 0.9906067415730337
---------
TN: 65553
FN: 125
TP: 66693
FP: 1129
---------
Accuracy der Validationsmenge innerhalb der bekannten Menge: 0.8595151515151516
Accuracy:0.8595151515151516
---------
TN: 6521
FN: 521
TP: 7661
FP: 1797
---------
Accuracy auf der unbekannten Menge: 0.837922077922078
time_to_prediction:0.6712961196899414
Accuracy auf der vollständigen gemischten Menge: 0.879638
Accuracy:0.879638
---------
TN: 209351
FN: 19532
TP: 230468
FP: 40649
---------
Sicherheitstest..:
google.de: ['ok']
1111-c111-1b11-a11d.reporo.net: ['malicious']


## Blacklist erstellen

In [10]:
#Blacklist erstellen. Jeder maliziöse Inhalt des Ursprungsdatensatzes wird zu einem Eintrag.
dataset_candidates, dataset_candidates_y = reduce_dataset(X, y, bekannte_Anzahl,0)
testset_x, testset_y = reduce_dataset(X, y, unbekannte_Anzahl,bekannte_Anzahl)

#print(dataset_candidates[:10])
#print(dataset_candidates_y[:10])
print("Length Trainingset:" + str(len(dataset_candidates)))
print("Length Testset:" + str(len(testset_x)))

#Dict wird verwendet, um die spätere Blacklistverwendung zu beschleunigen
start_time_blacklist_creation = time.time()
blacklist = dict()
for i in range(len(dataset_candidates_y)):
    if dataset_candidates_y[i] == "malicious":
        blacklist[dataset_candidates[i]] = 'malicious'

end_time_blacklist_creation = time.time()
time_blacklist_creation = end_time_blacklist_creation - start_time_blacklist_creation 
print("time_blacklist_creation:" + str(time_blacklist_creation))

print("Length Trainingset:" + str(len(blacklist)))

Length Trainingset:150000
Length Testset:350000
time_blacklist_creation:0.0352482795715332
Length Trainingset:75000


### Blacklist Test durchführen

In [11]:
#Auf bekannter Datenmenge evaluieren
blacklist_accuracy_history = list()
TP = 0
TN = 0
FP = 0
FN = 0
for i in range(len(dataset_candidates)):
    domain_name = dataset_candidates[i]
    predicted_is_malicious = domain_is_malicious(blacklist, domain_name)
    real_is_malicious = dataset_candidates_y[i]
    if real_is_malicious == "malicious":
        real_is_malicious = True
    else:
        real_is_malicious = False
    
    
    
    if real_is_malicious == True:
        if predicted_is_malicious == True:
            TP += 1
        elif predicted_is_malicious == False:
            FN += 1
    elif real_is_malicious == False:
        if predicted_is_malicious == True:
            FP += 1   
            print(domain_name)
        elif predicted_is_malicious == False:
            TN += 1

print("TP: " + str(TP))
print("TN: " + str(TN))
print("FP: " + str(FP))
print("FN: " + str(FN))
accuracy = (TP + TN) / (TP + TN + FP + FN)
print("Accuracy auf der bekannten Datenmenge: " + str(accuracy))


#Auf unbekannter Datenmenge evaluieren
blacklist_accuracy_history = list()
TP = 0
TN = 0
FP = 0
FN = 0
for i in range(len(testset_x)):
    domain_name = testset_x[i]
    predicted_is_malicious = domain_is_malicious(blacklist, domain_name)
    real_is_malicious = testset_y[i]
    if real_is_malicious == "malicious":
        real_is_malicious = True
    else:
        real_is_malicious = False
    
    if real_is_malicious == True:
        if predicted_is_malicious == True:
            TP += 1
        elif predicted_is_malicious == False:
            FN += 1
    elif real_is_malicious == False:
        if predicted_is_malicious == True:
            FP += 1   
        elif predicted_is_malicious == False:
            TN += 1

print("TP: " + str(TP))
print("TN: " + str(TN))
print("FP: " + str(FP))
print("FN: " + str(FN))
accuracy = (TP + TN) / (TP + TN + FP + FN)
print("Accuracy auf der unbekannten Datenmenge: " + str(accuracy))

#Auf dem gemischten Datensatz evaluieren
blacklist_accuracy_history = list()
TP = 0
TN = 0
FP = 0
FN = 0
for i in range(len(X)):
    domain_name = X[i]
    predicted_is_malicious = domain_is_malicious(blacklist, domain_name)
    real_is_malicious = y[i]
    if real_is_malicious == "malicious":
        real_is_malicious = True
    else:
        real_is_malicious = False
    
    if real_is_malicious == True:
        if predicted_is_malicious == True:
            TP += 1
        elif predicted_is_malicious == False:
            FN += 1
    elif real_is_malicious == False:
        if predicted_is_malicious == True:
            FP += 1   
        elif predicted_is_malicious == False:
            TN += 1
            
#Time Test ohne Confusion-Matrix-Berechnung
start_time_blacklist_prediction = time.time()
blacklist_accuracy_history = list()

for i in range(len(X)):
    domain_name = X[i]
    predicted_is_malicious = domain_is_malicious(blacklist, domain_name)
    real_is_malicious = y[i]
    if real_is_malicious == "malicious":
        real_is_malicious = True
    else:
        real_is_malicious = False          
end_time_blacklist_prediction = time.time()
time_blacklist_prediction = end_time_blacklist_prediction - start_time_blacklist_prediction
print("time_blacklist_prediction:" + str(time_blacklist_prediction))    


print("TP: " + str(TP))
print("TN: " + str(TN))
print("FP: " + str(FP))
print("FN: " + str(FN))
accuracy = (TP + TN) / (TP + TN + FP + FN)
print("Accuracy auf der gemischten Datenmenge: " + str(accuracy))

TP: 75000
TN: 75000
FP: 0
FN: 0
Accuracy auf der bekannten Datenmenge: 1.0
TP: 0
TN: 175000
FP: 0
FN: 175000
Accuracy auf der unbekannten Datenmenge: 0.5
time_blacklist_prediction:0.2731785774230957
TP: 75000
TN: 250000
FP: 0
FN: 175000
Accuracy auf der gemischten Datenmenge: 0.65


### Kombination Blacklist - Random Forest Modell testen

In [12]:
#N-Gram Vektorisierung durchführen
vectorized_testset_x, vectorizer = vectorize(n_gram_min, n_gram_max, ["0-100-195.btcc.com","google.de"], max_param_amount, vectorizer)
RANDOMFOREST_y_pred = classifier.predict(vectorized_testset_x)
print("Testprädiktion von 0-100-195.btcc.com und google.de:")
print(RANDOMFOREST_y_pred)

Testprädiktion von 0-100-195.btcc.com und google.de:
['malicious' 'ok']


In [13]:
X_rf_filtered_numbers = filter_numbers(X)
dataset_candidates_rf_filtered, dataset_candidates_y_rf_filtered = reduce_dataset(X_rf_filtered_numbers, y, bekannte_Anzahl,0)
testset_x_rf_filtered, testset_y_rf_filtered = reduce_dataset(X_rf_filtered_numbers, y, unbekannte_Anzahl,bekannte_Anzahl)

dataset_candidates, dataset_candidates_y = reduce_dataset(X, y, bekannte_Anzahl,0)
testset_x, testset_y = reduce_dataset(X, y, unbekannte_Anzahl,bekannte_Anzahl)


print("---------")

#N-Gram Vektorisierung durchführen
vectorized_testset_x, vectorizer = vectorize(n_gram_min, n_gram_max, dataset_candidates_rf_filtered, max_param_amount, vectorizer)
RANDOMFOREST_y_pred = classifier.predict(vectorized_testset_x)

TP = 0
FN = 0
FP = 0
TN = 0
#Zunächst überprüfen, ob Eintrag in Blacklist vorhanden ist.
for i in range(len(dataset_candidates)):
    domain_name = dataset_candidates[i]
    real_is_malicious = dataset_candidates_y[i]
    RANDOMFOREST_prediction = RANDOMFOREST_y_pred[i]
    BLACKLIST_predicted_is_malicious = domain_is_malicious(blacklist, domain_name)
    if real_is_malicious == "malicious":
        real_is_malicious = True
    else:
        real_is_malicious = False
        
    #Ist nicht auf der Liste. Also das RF-Modell befragen
    if BLACKLIST_predicted_is_malicious == False:
        #0 = maliziös, 1 = ok
        if RANDOMFOREST_prediction == "malicious": 
            combined_predicted_is_malicious = True
        else:
            combined_predicted_is_malicious = False
    else:
        combined_predicted_is_malicious = True

    if real_is_malicious == True:
        if combined_predicted_is_malicious == True:
            TP += 1
        elif combined_predicted_is_malicious == False:
            FN += 1
    elif real_is_malicious == False:
        if combined_predicted_is_malicious == True:
            FP += 1   
        elif combined_predicted_is_malicious == False:
            TN += 1

accuracy = (TP + TN) / (TP + TN + FP + FN)
print("TP: " + str(TP))
print("TN: " + str(TN))
print("FP: " + str(FP))
print("FN: " + str(FN))

#print("Anteil der : " + str(accuracy))
print("Accuracy der Trainingsmenge innerhalb der bekannten Daten: " + str(accuracy))


print("---------")

#N-Gram Vektorisierung durchführen
vectorized_testset_x, vectorizer = vectorize(n_gram_min, n_gram_max, testset_x_rf_filtered, max_param_amount, vectorizer)
RANDOMFOREST_y_pred = classifier.predict(vectorized_testset_x)

TP = 0
FN = 0
FP = 0
TN = 0
#Zunächst überprüfen, ob Eintrag in Blacklist vorhanden ist.
for i in range(len(testset_x)):
    domain_name = testset_x[i]
    real_is_malicious = testset_y[i]
    RANDOMFOREST_prediction = RANDOMFOREST_y_pred[i]
    BLACKLIST_predicted_is_malicious = domain_is_malicious(blacklist, domain_name)
    if real_is_malicious == "malicious":
        real_is_malicious = True
    else:
        real_is_malicious = False
        
    #Ist nicht auf der Liste. Also das RF-Modell befragen
    if BLACKLIST_predicted_is_malicious == False:
        #0 = maliziös, 1 = ok
        if RANDOMFOREST_prediction == "malicious": 
            combined_predicted_is_malicious = True
        else:
            combined_predicted_is_malicious = False
    else:
        combined_predicted_is_malicious = True

    if real_is_malicious == True:
        if combined_predicted_is_malicious == True:
            TP += 1
        elif combined_predicted_is_malicious == False:
            FN += 1
    elif real_is_malicious == False:
        if combined_predicted_is_malicious == True:
            FP += 1   
        elif combined_predicted_is_malicious == False:
            TN += 1

accuracy = (TP + TN) / (TP + TN + FP + FN)
print("TP: " + str(TP))
print("TN: " + str(TN))
print("FP: " + str(FP))
print("FN: " + str(FN))

#print("Anteil der : " + str(accuracy))
print("Accuracy der Validationsmenge innerhalb der bekannten Daten: " + str(accuracy))



print("---------")

#N-Gram Vektorisierung durchführen
vectorized_testset_x, vectorizer = vectorize(n_gram_min, n_gram_max, X_rf_filtered_numbers, max_param_amount, vectorizer)
RANDOMFOREST_y_pred = classifier.predict(vectorized_testset_x)

TP = 0
FN = 0
FP = 0
TN = 0
#Zunächst überprüfen, ob Eintrag in Blacklist vorhanden ist.
for i in range(len(X)):
    domain_name = X[i]
    real_is_malicious = y[i]
    RANDOMFOREST_prediction = RANDOMFOREST_y_pred[i]
    BLACKLIST_predicted_is_malicious = domain_is_malicious(blacklist, domain_name)
    if real_is_malicious == "malicious":
        real_is_malicious = True
    else:
        real_is_malicious = False
        
    #Ist nicht auf der Liste. Also das RF-Modell befragen
    if BLACKLIST_predicted_is_malicious == False:
        #0 = maliziös, 1 = ok
        if RANDOMFOREST_prediction == "malicious": 
            combined_predicted_is_malicious = True
        else:
            combined_predicted_is_malicious = False
    else:
        combined_predicted_is_malicious = True

    if real_is_malicious == True:
        if combined_predicted_is_malicious == True:
            TP += 1
        elif combined_predicted_is_malicious == False:
            FN += 1
    elif real_is_malicious == False:
        if combined_predicted_is_malicious == True:
            FP += 1   
        elif combined_predicted_is_malicious == False:
            TN += 1

accuracy = (TP + TN) / (TP + TN + FP + FN)
print("TP: " + str(TP))
print("TN: " + str(TN))
print("FP: " + str(FP))
print("FN: " + str(FN))

#print("Anteil der : " + str(accuracy))
print("Accuracy der gemischten Datenmenge: " + str(accuracy))

---------
TP: 75000
TN: 74354
FP: 646
FN: 0
Accuracy der Trainingsmenge innerhalb der bekannten Daten: 0.9956933333333333
---------
TP: 137277
TN: 156114
FP: 18886
FN: 37723
Accuracy der Validationsmenge innerhalb der bekannten Daten: 0.83826
---------
TP: 212277
TN: 230468
FP: 19532
FN: 37723
Accuracy der gemischten Datenmenge: 0.88549


## Szenario 2: 50% Ursprungsdatenmenge, 50% Testdatenmenge

### Ursprungsdatenmenge und Testdatenmenge aufteilen

In [14]:
groesse_der_bekannten_menge = 0.5

print("Vollständige Datensatzanzahl: " + str(len(X)))
Vollstaendige_Anzahl = len(X)
bekannte_Anzahl = round(Vollstaendige_Anzahl*groesse_der_bekannten_menge)
unbekannte_Anzahl = Vollstaendige_Anzahl - bekannte_Anzahl

Vollständige Datensatzanzahl: 500000


## Zufallsverfahren

In [15]:
#Auf dem gemischten Datensatz evaluieren
TP = 0
TN = 0
FP = 0
FN = 0
for i in range(len(X)):
    predicted_is_malicious = random.choice([True, False])
    real_is_malicious = y[i]
    if real_is_malicious == "malicious":
        real_is_malicious = True
    else:
        real_is_malicious = False
    
    if real_is_malicious == True:
        if predicted_is_malicious == True:
            TP += 1
        elif predicted_is_malicious == False:
            FN += 1
    elif real_is_malicious == False:
        if predicted_is_malicious == True:
            FP += 1   
        elif predicted_is_malicious == False:
            TN += 1
 
print("TP: " + str(TP))
print("TN: " + str(TN))
print("FP: " + str(FP))
print("FN: " + str(FN))
accuracy = (TP + TN) / (TP + TN + FP + FN)
print("Accuracy auf der gemischten Datenmenge: " + str(accuracy))

TP: 125362
TN: 124940
FP: 125060
FN: 124638
Accuracy auf der gemischten Datenmenge: 0.500604


### Random Forest Test durchführen

In [16]:
X_rf_filtered_numbers = filter_numbers(X)
Xbekannt, ybekannt = reduce_dataset(X_rf_filtered_numbers, y, bekannte_Anzahl,0)
Xunbekannt, yunbekannt = reduce_dataset(X_rf_filtered_numbers, y, unbekannte_Anzahl,bekannte_Anzahl)
print("Anzahl bekannter zu unbekannter Datensätze:")
print(str(len(Xbekannt)))
print(str(len(Xunbekannt)))
#Training nur innerhalb der Ursprungsdatenmenge mit der zuvor ermittelten Test-Size-Größe
vectorized_Xbekannt, vectorizer = vectorize(n_gram_min, n_gram_max, Xbekannt, max_param_amount)
X_train, X_test, y_train, y_test = train_test_split(vectorized_Xbekannt, ybekannt, test_size=test_size_percentage, random_state=0)
classifier = RandomForestClassifier(n_estimators=tree_count, random_state=0,n_jobs=-1)

start_time_fit = time.time()
classifier.fit(X_train, y_train) 
end_time_fit = time.time()
time_fit = end_time_fit - start_time_fit 
print("time_to_fit:" + str(time_fit))
print("---------")

y_pred = classifier.predict(X_train)
accuracy = accuracy_score(y_train, y_pred)
print("Accuracy der Trainingsmenge innerhalb der bekannten Menge: " + str(accuracy))
CM = confusion_matrix(y_train, y_pred)

print("---------")
TN = CM[0][0]
FN = CM[1][0]
TP = CM[1][1]
FP = CM[0][1]
print("TN: " + str(TN))
print("FN: " + str(FN))
print("TP: " + str(TP))
print("FP: " + str(FP))

print("---------")

y_pred = classifier.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy der Validationsmenge innerhalb der bekannten Menge: " + str(accuracy))
CM = confusion_matrix(y_test, y_pred)
print("Accuracy:" + str(accuracy))
print("---------")
TN = CM[0][0]
FN = CM[1][0]
TP = CM[1][1]
FP = CM[0][1]
print("TN: " + str(TN))
print("FN: " + str(FN))
print("TP: " + str(TP))
print("FP: " + str(FP))

print("---------")

#Unbekannte Menge bearbeiten..
vectorized_Xunbekannt, vectorizer = vectorize(n_gram_min, n_gram_max, Xunbekannt, max_param_amount, vectorizer)
X_train, X_test, y_train, y_test = train_test_split(vectorized_Xunbekannt, yunbekannt, test_size=test_size_percentage, random_state=0)
y_pred = classifier.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy auf der unbekannten Menge: " + str(accuracy))

#Memory freigeben
del vectorized_Xbekannt, vectorized_Xunbekannt, X_train, X_test, y_train, y_test, y_pred

#Beides zusammen bearbeiten für das zu ermittelnde Ergebnis
vectorized_Xmix, vectorizer = vectorize(n_gram_min, n_gram_max, X_rf_filtered_numbers, max_param_amount, vectorizer)
#X_train, X_test, y_train, y_test = train_test_split(vectorized_Xmix, y, test_size=test_size_percentage, random_state=0)

start_time_predict = time.time()
y_pred = classifier.predict(vectorized_Xmix)
end_time_predict = time.time()
time_predict = end_time_predict - start_time_predict 
print("time_to_prediction:" + str(time_predict))

accuracy = accuracy_score(y, y_pred)
print("Accuracy auf der vollständigen gemischten Menge: " + str(accuracy))
CM = confusion_matrix(y, y_pred)
print("Accuracy:" + str(accuracy))
print("---------")
TN = CM[0][0]
FN = CM[1][0]
TP = CM[1][1]
FP = CM[0][1]
print("TN: " + str(TN))
print("FN: " + str(FN))
print("TP: " + str(TP))
print("FP: " + str(FP))
print("---------")

print("Sicherheitstest..:")
vectorized_google, vectorizer = vectorize(n_gram_min, n_gram_max, ["google.de"], max_param_amount, vectorizer)
print("google.de: " + str(classifier.predict(vectorized_google)))
vectorized_1111, vectorizer = vectorize(n_gram_min, n_gram_max, ["1111-c111-1b11-a11d.reporo.net"], max_param_amount, vectorizer)
print("1111-c111-1b11-a11d.reporo.net: " + str(classifier.predict(vectorized_1111)))

Anzahl bekannter zu unbekannter Datensätze:
250000
250000
time_to_fit:53.68382668495178
---------
Accuracy der Trainingsmenge innerhalb der bekannten Menge: 0.9909393258426966
---------
TN: 109390
FN: 203
TP: 111094
FP: 1813
---------
Accuracy der Validationsmenge innerhalb der bekannten Menge: 0.8588363636363636
Accuracy:0.8588363636363636
---------
TN: 10822
FN: 907
TP: 12796
FP: 2975
---------
Accuracy auf der unbekannten Menge: 0.8476727272727272
time_to_prediction:0.8252668380737305
Accuracy auf der vollständigen gemischten Menge: 0.91208
Accuracy:0.91208
---------
TN: 218447
FN: 12407
TP: 237593
FP: 31553
---------
Sicherheitstest..:
google.de: ['ok']
1111-c111-1b11-a11d.reporo.net: ['malicious']


## Blacklist erstellen

In [17]:
#Blacklist erstellen. Jeder maliziöse Inhalt des Ursprungsdatensatzes wird zu einem Eintrag.
dataset_candidates, dataset_candidates_y = reduce_dataset(X, y, bekannte_Anzahl,0)
testset_x, testset_y = reduce_dataset(X, y, unbekannte_Anzahl,bekannte_Anzahl)

#print(dataset_candidates[:10])
#print(dataset_candidates_y[:10])
print("Length Trainingset:" + str(len(dataset_candidates)))
print("Length Testset:" + str(len(testset_x)))

#Dict wird verwendet, um die spätere Blacklistverwendung zu beschleunigen
start_time_blacklist_creation = time.time()
blacklist = dict()
for i in range(len(dataset_candidates_y)):
    if dataset_candidates_y[i] == "malicious":
        blacklist[dataset_candidates[i]] = 'malicious'

end_time_blacklist_creation = time.time()
time_blacklist_creation = end_time_blacklist_creation - start_time_blacklist_creation 
print("time_blacklist_creation:" + str(time_blacklist_creation))

print("Length Trainingset:" + str(len(blacklist)))

Length Trainingset:250000
Length Testset:250000
time_blacklist_creation:0.055199623107910156
Length Trainingset:125000


### Blacklist Test durchführen

In [18]:
#Auf bekannter Datenmenge evaluieren
blacklist_accuracy_history = list()
TP = 0
TN = 0
FP = 0
FN = 0
for i in range(len(dataset_candidates)):
    domain_name = dataset_candidates[i]
    predicted_is_malicious = domain_is_malicious(blacklist, domain_name)
    real_is_malicious = dataset_candidates_y[i]
    if real_is_malicious == "malicious":
        real_is_malicious = True
    else:
        real_is_malicious = False
    
    
    
    if real_is_malicious == True:
        if predicted_is_malicious == True:
            TP += 1
        elif predicted_is_malicious == False:
            FN += 1
    elif real_is_malicious == False:
        if predicted_is_malicious == True:
            FP += 1   
            print(domain_name)
        elif predicted_is_malicious == False:
            TN += 1

print("TP: " + str(TP))
print("TN: " + str(TN))
print("FP: " + str(FP))
print("FN: " + str(FN))
accuracy = (TP + TN) / (TP + TN + FP + FN)
print("Accuracy auf der bekannten Datenmenge: " + str(accuracy))


#Auf unbekannter Datenmenge evaluieren
blacklist_accuracy_history = list()
TP = 0
TN = 0
FP = 0
FN = 0
for i in range(len(testset_x)):
    domain_name = testset_x[i]
    predicted_is_malicious = domain_is_malicious(blacklist, domain_name)
    real_is_malicious = testset_y[i]
    if real_is_malicious == "malicious":
        real_is_malicious = True
    else:
        real_is_malicious = False
    
    if real_is_malicious == True:
        if predicted_is_malicious == True:
            TP += 1
        elif predicted_is_malicious == False:
            FN += 1
    elif real_is_malicious == False:
        if predicted_is_malicious == True:
            FP += 1   
        elif predicted_is_malicious == False:
            TN += 1

print("TP: " + str(TP))
print("TN: " + str(TN))
print("FP: " + str(FP))
print("FN: " + str(FN))
accuracy = (TP + TN) / (TP + TN + FP + FN)
print("Accuracy auf der unbekannten Datenmenge: " + str(accuracy))

#Auf dem gemischten Datensatz evaluieren
blacklist_accuracy_history = list()
TP = 0
TN = 0
FP = 0
FN = 0
for i in range(len(X)):
    domain_name = X[i]
    predicted_is_malicious = domain_is_malicious(blacklist, domain_name)
    real_is_malicious = y[i]
    if real_is_malicious == "malicious":
        real_is_malicious = True
    else:
        real_is_malicious = False
    
    if real_is_malicious == True:
        if predicted_is_malicious == True:
            TP += 1
        elif predicted_is_malicious == False:
            FN += 1
    elif real_is_malicious == False:
        if predicted_is_malicious == True:
            FP += 1   
        elif predicted_is_malicious == False:
            TN += 1

#Time Test ohne Confusion-Matrix-Berechnung
start_time_blacklist_prediction = time.time()
blacklist_accuracy_history = list()
for i in range(len(X)):
    domain_name = X[i]
    predicted_is_malicious = domain_is_malicious(blacklist, domain_name)
    real_is_malicious = y[i]
    if real_is_malicious == "malicious":
        real_is_malicious = True
    else:
        real_is_malicious = False          
end_time_blacklist_prediction = time.time()
time_blacklist_prediction = end_time_blacklist_prediction - start_time_blacklist_prediction
print("time_blacklist_prediction:" + str(time_blacklist_prediction))            
            
                
            
print("TP: " + str(TP))
print("TN: " + str(TN))
print("FP: " + str(FP))
print("FN: " + str(FN))
accuracy = (TP + TN) / (TP + TN + FP + FN)
print("Accuracy auf der gemischten Datenmenge: " + str(accuracy))

TP: 125000
TN: 125000
FP: 0
FN: 0
Accuracy auf der bekannten Datenmenge: 1.0
TP: 0
TN: 125000
FP: 0
FN: 125000
Accuracy auf der unbekannten Datenmenge: 0.5
time_blacklist_prediction:0.26996445655822754
TP: 125000
TN: 250000
FP: 0
FN: 125000
Accuracy auf der gemischten Datenmenge: 0.75


### Kombination Blacklist - Random Forest Modell testen

In [19]:
#N-Gram Vektorisierung durchführen
vectorized_testset_x, vectorizer = vectorize(n_gram_min, n_gram_max, ["0-100-195.btcc.com","google.de"], max_param_amount, vectorizer)
RANDOMFOREST_y_pred = classifier.predict(vectorized_testset_x)
print("Testprädiktion von 0-100-195.btcc.com und google.de:")
print(RANDOMFOREST_y_pred)

Testprädiktion von 0-100-195.btcc.com und google.de:
['malicious' 'ok']


In [20]:
X_rf_filtered_numbers = filter_numbers(X)
dataset_candidates_rf_filtered, dataset_candidates_y_rf_filtered = reduce_dataset(X_rf_filtered_numbers, y, bekannte_Anzahl,0)
testset_x_rf_filtered, testset_y_rf_filtered = reduce_dataset(X_rf_filtered_numbers, y, unbekannte_Anzahl,bekannte_Anzahl)

dataset_candidates, dataset_candidates_y = reduce_dataset(X, y, bekannte_Anzahl,0)
testset_x, testset_y = reduce_dataset(X, y, unbekannte_Anzahl,bekannte_Anzahl)


print("---------")

#N-Gram Vektorisierung durchführen
vectorized_testset_x, vectorizer = vectorize(n_gram_min, n_gram_max, dataset_candidates_rf_filtered, max_param_amount, vectorizer)
RANDOMFOREST_y_pred = classifier.predict(vectorized_testset_x)

TP = 0
FN = 0
FP = 0
TN = 0
#Zunächst überprüfen, ob Eintrag in Blacklist vorhanden ist.
for i in range(len(dataset_candidates)):
    domain_name = dataset_candidates[i]
    real_is_malicious = dataset_candidates_y[i]
    RANDOMFOREST_prediction = RANDOMFOREST_y_pred[i]
    BLACKLIST_predicted_is_malicious = domain_is_malicious(blacklist, domain_name)
    if real_is_malicious == "malicious":
        real_is_malicious = True
    else:
        real_is_malicious = False
        
    #Ist nicht auf der Liste. Also das RF-Modell befragen
    if BLACKLIST_predicted_is_malicious == False:
        #0 = maliziös, 1 = ok
        if RANDOMFOREST_prediction == "malicious": 
            combined_predicted_is_malicious = True
        else:
            combined_predicted_is_malicious = False
    else:
        combined_predicted_is_malicious = True

    if real_is_malicious == True:
        if combined_predicted_is_malicious == True:
            TP += 1
        elif combined_predicted_is_malicious == False:
            FN += 1
    elif real_is_malicious == False:
        if combined_predicted_is_malicious == True:
            FP += 1   
        elif combined_predicted_is_malicious == False:
            TN += 1

accuracy = (TP + TN) / (TP + TN + FP + FN)
print("TP: " + str(TP))
print("TN: " + str(TN))
print("FP: " + str(FP))
print("FN: " + str(FN))

#print("Anteil der : " + str(accuracy))
print("Accuracy der Trainingsmenge innerhalb der bekannten Daten: " + str(accuracy))


print("---------")

#N-Gram Vektorisierung durchführen
vectorized_testset_x, vectorizer = vectorize(n_gram_min, n_gram_max, testset_x_rf_filtered, max_param_amount, vectorizer)
RANDOMFOREST_y_pred = classifier.predict(vectorized_testset_x)

TP = 0
FN = 0
FP = 0
TN = 0
#Zunächst überprüfen, ob Eintrag in Blacklist vorhanden ist.
for i in range(len(testset_x)):
    domain_name = testset_x[i]
    real_is_malicious = testset_y[i]
    RANDOMFOREST_prediction = RANDOMFOREST_y_pred[i]
    BLACKLIST_predicted_is_malicious = domain_is_malicious(blacklist, domain_name)
    if real_is_malicious == "malicious":
        real_is_malicious = True
    else:
        real_is_malicious = False
        
    #Ist nicht auf der Liste. Also das RF-Modell befragen
    if BLACKLIST_predicted_is_malicious == False:
        #0 = maliziös, 1 = ok
        if RANDOMFOREST_prediction == "malicious": 
            combined_predicted_is_malicious = True
        else:
            combined_predicted_is_malicious = False
    else:
        combined_predicted_is_malicious = True

    if real_is_malicious == True:
        if combined_predicted_is_malicious == True:
            TP += 1
        elif combined_predicted_is_malicious == False:
            FN += 1
    elif real_is_malicious == False:
        if combined_predicted_is_malicious == True:
            FP += 1   
        elif combined_predicted_is_malicious == False:
            TN += 1

accuracy = (TP + TN) / (TP + TN + FP + FN)
print("TP: " + str(TP))
print("TN: " + str(TN))
print("FP: " + str(FP))
print("FN: " + str(FN))

#print("Anteil der : " + str(accuracy))
print("Accuracy der Validationsmenge innerhalb der bekannten Daten: " + str(accuracy))



print("---------")

#N-Gram Vektorisierung durchführen
vectorized_testset_x, vectorizer = vectorize(n_gram_min, n_gram_max, X_rf_filtered_numbers, max_param_amount, vectorizer)
RANDOMFOREST_y_pred = classifier.predict(vectorized_testset_x)

TP = 0
FN = 0
FP = 0
TN = 0
#Zunächst überprüfen, ob Eintrag in Blacklist vorhanden ist.
for i in range(len(X)):
    domain_name = X[i]
    real_is_malicious = y[i]
    RANDOMFOREST_prediction = RANDOMFOREST_y_pred[i]
    BLACKLIST_predicted_is_malicious = domain_is_malicious(blacklist, domain_name)
    if real_is_malicious == "malicious":
        real_is_malicious = True
    else:
        real_is_malicious = False
        
    #Ist nicht auf der Liste. Also das RF-Modell befragen
    if BLACKLIST_predicted_is_malicious == False:
        #0 = maliziös, 1 = ok
        if RANDOMFOREST_prediction == "malicious": 
            combined_predicted_is_malicious = True
        else:
            combined_predicted_is_malicious = False
    else:
        combined_predicted_is_malicious = True

    if real_is_malicious == True:
        if combined_predicted_is_malicious == True:
            TP += 1
        elif combined_predicted_is_malicious == False:
            FN += 1
    elif real_is_malicious == False:
        if combined_predicted_is_malicious == True:
            FP += 1   
        elif combined_predicted_is_malicious == False:
            TN += 1

accuracy = (TP + TN) / (TP + TN + FP + FN)
print("TP: " + str(TP))
print("TN: " + str(TN))
print("FP: " + str(FP))
print("FN: " + str(FN))

#print("Anteil der : " + str(accuracy))
print("Accuracy der gemischten Datenmenge: " + str(accuracy))

---------
TP: 125000
TN: 123890
FP: 1110
FN: 0
Accuracy der Trainingsmenge innerhalb der bekannten Daten: 0.99556
---------
TP: 98235
TN: 113703
FP: 11297
FN: 26765
Accuracy der Validationsmenge innerhalb der bekannten Daten: 0.847752
---------
TP: 223235
TN: 237593
FP: 12407
FN: 26765
Accuracy der gemischten Datenmenge: 0.921656


## Szenario 3: 80% Ursprungsdatenmenge, 20% Testdatenmenge

### Ursprungsdatenmenge und Testdatenmenge aufteilen

In [21]:
groesse_der_bekannten_menge = 0.8

print("Vollständige Datensatzanzahl: " + str(len(X)))
Vollstaendige_Anzahl = len(X)
bekannte_Anzahl = round(Vollstaendige_Anzahl*groesse_der_bekannten_menge)
unbekannte_Anzahl = Vollstaendige_Anzahl - bekannte_Anzahl

Vollständige Datensatzanzahl: 500000


## Zufallsverfahren

In [22]:
#Auf dem gemischten Datensatz evaluieren
TP = 0
TN = 0
FP = 0
FN = 0
for i in range(len(X)):
    predicted_is_malicious = random.choice([True, False])
    real_is_malicious = y[i]
    if real_is_malicious == "malicious":
        real_is_malicious = True
    else:
        real_is_malicious = False
    
    if real_is_malicious == True:
        if predicted_is_malicious == True:
            TP += 1
        elif predicted_is_malicious == False:
            FN += 1
    elif real_is_malicious == False:
        if predicted_is_malicious == True:
            FP += 1   
        elif predicted_is_malicious == False:
            TN += 1
 
print("TP: " + str(TP))
print("TN: " + str(TN))
print("FP: " + str(FP))
print("FN: " + str(FN))
accuracy = (TP + TN) / (TP + TN + FP + FN)
print("Accuracy auf der gemischten Datenmenge: " + str(accuracy))

TP: 125035
TN: 124986
FP: 125014
FN: 124965
Accuracy auf der gemischten Datenmenge: 0.500042


### Random Forest Test durchführen

In [23]:
X_rf_filtered_numbers = filter_numbers(X)
Xbekannt, ybekannt = reduce_dataset(X_rf_filtered_numbers, y, bekannte_Anzahl,0)
Xunbekannt, yunbekannt = reduce_dataset(X_rf_filtered_numbers, y, unbekannte_Anzahl,bekannte_Anzahl)
print("Anzahl bekannter zu unbekannter Datensätze:")
print(str(len(Xbekannt)))
print(str(len(Xunbekannt)))
#Training nur innerhalb der Ursprungsdatenmenge mit der zuvor ermittelten Test-Size-Größe
vectorized_Xbekannt, vectorizer = vectorize(n_gram_min, n_gram_max, Xbekannt, max_param_amount)
X_train, X_test, y_train, y_test = train_test_split(vectorized_Xbekannt, ybekannt, test_size=test_size_percentage, random_state=0)
classifier = RandomForestClassifier(n_estimators=tree_count, random_state=0,n_jobs=-1)

start_time_rf_fit = time.time()
classifier.fit(X_train, y_train) 
end_time_rf_fit = time.time()
time_rf_fit = end_time_rf_fit - start_time_rf_fit 
print("time_rf_fit:" + str(time_rf_fit))   

print("---------")

y_pred = classifier.predict(X_train)
accuracy = accuracy_score(y_train, y_pred)
print("Accuracy der Trainingsmenge innerhalb der bekannten Menge: " + str(accuracy))
CM = confusion_matrix(y_train, y_pred)

print("---------")
TN = CM[0][0]
FN = CM[1][0]
TP = CM[1][1]
FP = CM[0][1]
print("TN: " + str(TN))
print("FN: " + str(FN))
print("TP: " + str(TP))
print("FP: " + str(FP))

print("---------")

y_pred = classifier.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy der Validationsmenge innerhalb der bekannten Menge: " + str(accuracy))
CM = confusion_matrix(y_test, y_pred)
print("Accuracy:" + str(accuracy))
print("---------")
TN = CM[0][0]
FN = CM[1][0]
TP = CM[1][1]
FP = CM[0][1]
print("TN: " + str(TN))
print("FN: " + str(FN))
print("TP: " + str(TP))
print("FP: " + str(FP))

print("---------")

#Unbekannte Menge bearbeiten..
vectorized_Xunbekannt, vectorizer = vectorize(n_gram_min, n_gram_max, Xunbekannt, max_param_amount, vectorizer)
X_train, X_test, y_train, y_test = train_test_split(vectorized_Xunbekannt, yunbekannt, test_size=test_size_percentage, random_state=0)
y_pred = classifier.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy auf der unbekannten Menge: " + str(accuracy))

#Memory freigeben
del vectorized_Xbekannt, vectorized_Xunbekannt, X_train, X_test, y_train, y_test, y_pred

#Beides zusammen bearbeiten für das zu ermittelnde Ergebnis
vectorized_Xmix, vectorizer = vectorize(n_gram_min, n_gram_max, X_rf_filtered_numbers, max_param_amount, vectorizer)
#X_train, X_test, y_train, y_test = train_test_split(vectorized_Xmix, y, test_size=test_size_percentage, random_state=0)


start_time_rf_prediction = time.time()
y_pred = classifier.predict(vectorized_Xmix)
end_time_rf_prediction = time.time()
time_rf_prediction = end_time_rf_prediction - start_time_rf_prediction
print("time_rf_prediction:" + str(time_rf_prediction))   

accuracy = accuracy_score(y, y_pred)
print("Accuracy auf der vollständigen gemischten Menge: " + str(accuracy))
CM = confusion_matrix(y, y_pred)
print("Accuracy:" + str(accuracy))
print("---------")
TN = CM[0][0]
FN = CM[1][0]
TP = CM[1][1]
FP = CM[0][1]
print("TN: " + str(TN))
print("FN: " + str(FN))
print("TP: " + str(TP))
print("FP: " + str(FP))
print("---------")

print("Sicherheitstest..:")
vectorized_google, vectorizer = vectorize(n_gram_min, n_gram_max, ["google.de"], max_param_amount, vectorizer)
print("google.de: " + str(classifier.predict(vectorized_google)))
vectorized_1111, vectorizer = vectorize(n_gram_min, n_gram_max, ["1111-c111-1b11-a11d.reporo.net"], max_param_amount, vectorizer)
print("1111-c111-1b11-a11d.reporo.net: " + str(classifier.predict(vectorized_1111)))


TP_count = 5
FN_count = 5
FP_count = 5
counter = 0
TN_count = 5
limit_bekannte_menge = bekannte_Anzahl
#Zeige falsch vorhergesagte URLs aus bekannter Menge
print("beispiele aus bekannter menge:")
for input, prediction, label in zip(X, y_pred, y): 
    counter += 1
    if counter < limit_bekannte_menge:
        if label == "malicious":
            if prediction == "malicious" and TP_count > 0:
                TP_count -= 1
                print("TP: " + input)
            elif prediction == "ok" and FN_count > 0:
                FN_count -= 1
                print("FN: " + input)
        elif label == "ok":
            if prediction == "malicious" and FP_count > 0:
                FP_count -= 1
                print("FP: " + input)            
            elif prediction == "ok" and TN_count > 0:
                TN_count -= 1
                print("TN: " + input)  
            
    if (TP_count+FN_count+FP_count+TN_count) == 0:
        break

#Zeige falsch vorhergesagte URLs aus bekannter Menge
TP_count = 5
FN_count = 5
FP_count = 5
counter = 0
TN_count = 5
print("beispiele aus unbekannter menge:")
for input, prediction, label in zip(X, y_pred, y): 
    counter += 1
    if counter >= limit_bekannte_menge:
        if label == "malicious":
            if prediction == "malicious" and TP_count > 0:
                TP_count -= 1
                print("TP: " + input)
            elif prediction == "ok" and FN_count > 0:
                FN_count -= 1
                print("FN: " + input)
        elif label == "ok":
            if prediction == "malicious" and FP_count > 0:
                FP_count -= 1
                print("FP: " + input)            
            elif prediction == "ok" and TN_count > 0:
                TN_count -= 1
                print("TN: " + input)  
            
    if (TP_count+FN_count+FP_count+TN_count) == 0:
        break
        
    #if prediction != label:
    #if prediction == "malicious":        
        
        #print(input, 'has been classified as ', prediction, 'and should be ', label) 

Anzahl bekannter zu unbekannter Datensätze:
400000
100000
time_rf_fit:137.7752480506897
---------
Accuracy der Trainingsmenge innerhalb der bekannten Menge: 0.9906404494382023
---------
TN: 175114
FN: 297
TP: 177554
FP: 3035
---------
Accuracy der Validationsmenge innerhalb der bekannten Menge: 0.8615909090909091
Accuracy:0.8615909090909091
---------
TN: 17026
FN: 1265
TP: 20884
FP: 4825
---------
Accuracy auf der unbekannten Menge: 0.859
time_rf_prediction:0.815814733505249
Accuracy auf der vollständigen gemischten Menge: 0.95332
Accuracy:0.95332
---------
TN: 231434
FN: 4774
TP: 245226
FP: 18566
---------
Sicherheitstest..:
google.de: ['ok']
1111-c111-1b11-a11d.reporo.net: ['malicious']
beispiele aus bekannter menge:
TP: www.api.binance.com
TN: google.com
TP: pool.burstcoin.ml
TN: youtube.com
TP: www.uupool.com
TN: tmall.com
TP: mollnia.com
TN: baidu.com
TP: wn-iad02.e-planning.net
TN: qq.com
FN: canadianmedstores.org
FN: sivaayavasi.org
FN: wansernmy.com
FN: mluiza.com
FN: webbizwil

## Blacklist erstellen

In [24]:
#Blacklist erstellen. Jeder maliziöse Inhalt des Ursprungsdatensatzes wird zu einem Eintrag.
dataset_candidates, dataset_candidates_y = reduce_dataset(X, y, bekannte_Anzahl,0)
testset_x, testset_y = reduce_dataset(X, y, unbekannte_Anzahl,bekannte_Anzahl)

#print(dataset_candidates[:10])
#print(dataset_candidates_y[:10])
print("Length Trainingset:" + str(len(dataset_candidates)))
print("Length Testset:" + str(len(testset_x)))

#Dict wird verwendet, um die spätere Blacklistverwendung zu beschleunigen
start_time_blacklist_creation = time.time()
blacklist = dict()
for i in range(len(dataset_candidates_y)):
    if dataset_candidates_y[i] == "malicious":
        blacklist[dataset_candidates[i]] = 'malicious'

end_time_blacklist_creation = time.time()
time_blacklist_creation = end_time_blacklist_creation - start_time_blacklist_creation
print("time_blacklist_creation:" + str(time_blacklist_creation))  
        
        
print("Length Trainingset:" + str(len(blacklist)))

Length Trainingset:400000
Length Testset:100000
time_blacklist_creation:0.10512995719909668
Length Trainingset:200000


### Blacklist Test durchführen

In [25]:
#Auf bekannter Datenmenge evaluieren
blacklist_accuracy_history = list()
TP = 0
TN = 0
FP = 0
FN = 0
for i in range(len(dataset_candidates)):
    domain_name = dataset_candidates[i]
    predicted_is_malicious = domain_is_malicious(blacklist, domain_name)
    real_is_malicious = dataset_candidates_y[i]
    if real_is_malicious == "malicious":
        real_is_malicious = True
    else:
        real_is_malicious = False
    
    
    
    if real_is_malicious == True:
        if predicted_is_malicious == True:
            TP += 1
        elif predicted_is_malicious == False:
            FN += 1
    elif real_is_malicious == False:
        if predicted_is_malicious == True:
            FP += 1   
            print(domain_name)
        elif predicted_is_malicious == False:
            TN += 1

print("TP: " + str(TP))
print("TN: " + str(TN))
print("FP: " + str(FP))
print("FN: " + str(FN))
accuracy = (TP + TN) / (TP + TN + FP + FN)
print("Accuracy auf der bekannten Datenmenge: " + str(accuracy))


#Auf unbekannter Datenmenge evaluieren
blacklist_accuracy_history = list()
TP = 0
TN = 0
FP = 0
FN = 0
for i in range(len(testset_x)):
    domain_name = testset_x[i]
    predicted_is_malicious = domain_is_malicious(blacklist, domain_name)
    real_is_malicious = testset_y[i]
    if real_is_malicious == "malicious":
        real_is_malicious = True
    else:
        real_is_malicious = False
    
    if real_is_malicious == True:
        if predicted_is_malicious == True:
            TP += 1
        elif predicted_is_malicious == False:
            FN += 1
    elif real_is_malicious == False:
        if predicted_is_malicious == True:
            FP += 1   
        elif predicted_is_malicious == False:
            TN += 1

print("TP: " + str(TP))
print("TN: " + str(TN))
print("FP: " + str(FP))
print("FN: " + str(FN))
accuracy = (TP + TN) / (TP + TN + FP + FN)
print("Accuracy auf der unbekannten Datenmenge: " + str(accuracy))

#Auf dem gemischten Datensatz evaluieren
blacklist_accuracy_history = list()
TP = 0
TN = 0
FP = 0
FN = 0
for i in range(len(X)):
    domain_name = X[i]
    predicted_is_malicious = domain_is_malicious(blacklist, domain_name)
    real_is_malicious = y[i]
    if real_is_malicious == "malicious":
        real_is_malicious = True
    else:
        real_is_malicious = False
    
    if real_is_malicious == True:
        if predicted_is_malicious == True:
            TP += 1
        elif predicted_is_malicious == False:
            FN += 1
    elif real_is_malicious == False:
        if predicted_is_malicious == True:
            FP += 1   
        elif predicted_is_malicious == False:
            TN += 1

            
            
#Time Test ohne Confusion-Matrix-Berechnung
start_time_blacklist_prediction = time.time()
blacklist_accuracy_history = list()

for i in range(len(X)):
    domain_name = X[i]
    predicted_is_malicious = domain_is_malicious(blacklist, domain_name)
    real_is_malicious = y[i]
    if real_is_malicious == "malicious":
        real_is_malicious = True
    else:
        real_is_malicious = False          
end_time_blacklist_prediction = time.time()
time_blacklist_prediction = end_time_blacklist_prediction - start_time_blacklist_prediction
print("time_blacklist_prediction:" + str(time_blacklist_prediction))             
            
print("TP: " + str(TP))
print("TN: " + str(TN))
print("FP: " + str(FP))
print("FN: " + str(FN))
accuracy = (TP + TN) / (TP + TN + FP + FN)
print("Accuracy auf der gemischten Datenmenge: " + str(accuracy))

TP: 200000
TN: 200000
FP: 0
FN: 0
Accuracy auf der bekannten Datenmenge: 1.0
TP: 0
TN: 50000
FP: 0
FN: 50000
Accuracy auf der unbekannten Datenmenge: 0.5
time_blacklist_prediction:0.2803795337677002
TP: 200000
TN: 250000
FP: 0
FN: 50000
Accuracy auf der gemischten Datenmenge: 0.9


### Kombination Blacklist - Random Forest Modell testen

In [26]:
#N-Gram Vektorisierung durchführen
vectorized_testset_x, vectorizer = vectorize(n_gram_min, n_gram_max, ["0-100-195.btcc.com","google.de"], max_param_amount, vectorizer)
RANDOMFOREST_y_pred = classifier.predict(vectorized_testset_x)
print("Testprädiktion von 0-100-195.btcc.com und google.de:")
print(RANDOMFOREST_y_pred)

Testprädiktion von 0-100-195.btcc.com und google.de:
['malicious' 'ok']


In [27]:
X_rf_filtered_numbers = filter_numbers(X)
dataset_candidates_rf_filtered, dataset_candidates_y_rf_filtered = reduce_dataset(X_rf_filtered_numbers, y, bekannte_Anzahl,0)
testset_x_rf_filtered, testset_y_rf_filtered = reduce_dataset(X_rf_filtered_numbers, y, unbekannte_Anzahl,bekannte_Anzahl)

dataset_candidates, dataset_candidates_y = reduce_dataset(X, y, bekannte_Anzahl,0)
testset_x, testset_y = reduce_dataset(X, y, unbekannte_Anzahl,bekannte_Anzahl)


print("---------")

#N-Gram Vektorisierung durchführen
vectorized_testset_x, vectorizer = vectorize(n_gram_min, n_gram_max, dataset_candidates_rf_filtered, max_param_amount, vectorizer)
RANDOMFOREST_y_pred = classifier.predict(vectorized_testset_x)

TP = 0
FN = 0
FP = 0
TN = 0
#Zunächst überprüfen, ob Eintrag in Blacklist vorhanden ist.
for i in range(len(dataset_candidates)):
    domain_name = dataset_candidates[i]
    real_is_malicious = dataset_candidates_y[i]
    RANDOMFOREST_prediction = RANDOMFOREST_y_pred[i]
    BLACKLIST_predicted_is_malicious = domain_is_malicious(blacklist, domain_name)
    if real_is_malicious == "malicious":
        real_is_malicious = True
    else:
        real_is_malicious = False
        
    #Ist nicht auf der Liste. Also das RF-Modell befragen
    if BLACKLIST_predicted_is_malicious == False:
        #0 = maliziös, 1 = ok
        if RANDOMFOREST_prediction == "malicious": 
            combined_predicted_is_malicious = True
        else:
            combined_predicted_is_malicious = False
    else:
        combined_predicted_is_malicious = True

    if real_is_malicious == True:
        if combined_predicted_is_malicious == True:
            TP += 1
        elif combined_predicted_is_malicious == False:
            FN += 1
    elif real_is_malicious == False:
        if combined_predicted_is_malicious == True:
            FP += 1   
        elif combined_predicted_is_malicious == False:
            TN += 1

accuracy = (TP + TN) / (TP + TN + FP + FN)
print("TP: " + str(TP))
print("TN: " + str(TN))
print("FP: " + str(FP))
print("FN: " + str(FN))

#print("Anteil der : " + str(accuracy))
print("Accuracy der Trainingsmenge innerhalb der bekannten Daten: " + str(accuracy))


print("---------")

#N-Gram Vektorisierung durchführen
vectorized_testset_x, vectorizer = vectorize(n_gram_min, n_gram_max, testset_x_rf_filtered, max_param_amount, vectorizer)
RANDOMFOREST_y_pred = classifier.predict(vectorized_testset_x)

TP = 0
FN = 0
FP = 0
TN = 0
#Zunächst überprüfen, ob Eintrag in Blacklist vorhanden ist.
for i in range(len(testset_x)):
    domain_name = testset_x[i]
    real_is_malicious = testset_y[i]
    RANDOMFOREST_prediction = RANDOMFOREST_y_pred[i]
    BLACKLIST_predicted_is_malicious = domain_is_malicious(blacklist, domain_name)
    if real_is_malicious == "malicious":
        real_is_malicious = True
    else:
        real_is_malicious = False
        
    #Ist nicht auf der Liste. Also das RF-Modell befragen
    if BLACKLIST_predicted_is_malicious == False:
        #0 = maliziös, 1 = ok
        if RANDOMFOREST_prediction == "malicious": 
            combined_predicted_is_malicious = True
        else:
            combined_predicted_is_malicious = False
    else:
        combined_predicted_is_malicious = True

    if real_is_malicious == True:
        if combined_predicted_is_malicious == True:
            TP += 1
        elif combined_predicted_is_malicious == False:
            FN += 1
    elif real_is_malicious == False:
        if combined_predicted_is_malicious == True:
            FP += 1   
        elif combined_predicted_is_malicious == False:
            TN += 1

accuracy = (TP + TN) / (TP + TN + FP + FN)
print("TP: " + str(TP))
print("TN: " + str(TN))
print("FP: " + str(FP))
print("FN: " + str(FN))

#print("Anteil der : " + str(accuracy))
print("Accuracy der Validationsmenge innerhalb der bekannten Daten: " + str(accuracy))



print("---------")

#N-Gram Vektorisierung durchführen
vectorized_testset_x, vectorizer = vectorize(n_gram_min, n_gram_max, X_rf_filtered_numbers, max_param_amount, vectorizer)
RANDOMFOREST_y_pred = classifier.predict(vectorized_testset_x)

TP = 0
FN = 0
FP = 0
TN = 0
#Zunächst überprüfen, ob Eintrag in Blacklist vorhanden ist.
for i in range(len(X)):
    domain_name = X[i]
    real_is_malicious = y[i]
    RANDOMFOREST_prediction = RANDOMFOREST_y_pred[i]
    BLACKLIST_predicted_is_malicious = domain_is_malicious(blacklist, domain_name)
    if real_is_malicious == "malicious":
        real_is_malicious = True
    else:
        real_is_malicious = False
        
    #Ist nicht auf der Liste. Also das RF-Modell befragen
    if BLACKLIST_predicted_is_malicious == False:
        #0 = maliziös, 1 = ok
        if RANDOMFOREST_prediction == "malicious": 
            combined_predicted_is_malicious = True
        else:
            combined_predicted_is_malicious = False
    else:
        combined_predicted_is_malicious = True

    if real_is_malicious == True:
        if combined_predicted_is_malicious == True:
            TP += 1
        elif combined_predicted_is_malicious == False:
            FN += 1
    elif real_is_malicious == False:
        if combined_predicted_is_malicious == True:
            FP += 1   
        elif combined_predicted_is_malicious == False:
            TN += 1

accuracy = (TP + TN) / (TP + TN + FP + FN)
print("TP: " + str(TP))
print("TN: " + str(TN))
print("FP: " + str(FP))
print("FN: " + str(FN))

#print("Anteil der : " + str(accuracy))
print("Accuracy der gemischten Datenmenge: " + str(accuracy))

---------
TP: 200000
TN: 198438
FP: 1562
FN: 0
Accuracy der Trainingsmenge innerhalb der bekannten Daten: 0.996095
---------
TP: 39294
TN: 46788
FP: 3212
FN: 10706
Accuracy der Validationsmenge innerhalb der bekannten Daten: 0.86082
---------
TP: 239294
TN: 245226
FP: 4774
FN: 10706
Accuracy der gemischten Datenmenge: 0.96904
